In [21]:
import praw
import pandas as pd
from datetime import datetime, timedelta
import time

In [23]:
reddit = praw.Reddit(
    client_id='',
    client_secret='',
    user_agent=''
)

In [25]:
# subreddits = ['PoliticalDiscussion', 'politics' ]
subreddits = ['PoliticalDiscussion']

In [27]:
start_date = datetime(2024, 7, 1)
end_date = datetime(2024, 11, 4, 23, 59, 59)

post_data = {
    'post_id': [], 'title': [], 'content': [], 'url': [], 'created_utc': [], 'subreddit': [],
    'num_comments': [], 'upvotes': [], 'upvote_ratio': [], 'author': []
}

comment_data = {
    'comment_id': [], 'post_title': [], 'comment_content': [], 'comment_created_utc': []
}

In [29]:
def handle_rate_limit():
    limits = reddit.auth.limits
    remaining_requests = limits['remaining']
    reset_time = limits['reset_timestamp'] - time.time()
    print(f"Remaining requests: {remaining_requests} | Resets in {reset_time:.2f} seconds.")
    
    if remaining_requests < 10:
        wait_time = max(reset_time + 10, 60)  # Add a buffer
        print(f"Rate limit reached. Sleeping for {wait_time:.2f} seconds.")
        time.sleep(wait_time)

In [31]:
for subreddit_name in subreddits:
    subreddit = reddit.subreddit(subreddit_name)
    post_count = 0
    
    for submission in subreddit.new(limit=None):  # Using `.new()` to get posts in descending order by time
        post_date = datetime.utcfromtimestamp(submission.created_utc)

        if post_date < start_date:
            break 
        if post_date > end_date:
            continue 

        post_data['post_id'].append(submission.id)
        post_data['title'].append(submission.title)
        # post_data['content'].append(submission.selftext)
        post_data['created_utc'].append(post_date)
        post_data['subreddit'].append(subreddit_name)
        post_data['num_comments'].append(submission.num_comments)
        post_data['upvotes'].append(submission.score)
        post_data['upvote_ratio'].append(submission.upvote_ratio)
        post_data['author'].append(submission.author.name if submission.author else 'Deleted')

        if submission.selftext:
            post_data['content'].append(submission.selftext)
            post_data['url'].append(submission.url if submission.url != submission.permalink else None) 
        else:
            post_data['content'].append("Link post")
            post_data['url'].append(submission.url)

        post_count += 1

        handle_rate_limit()
        
        time.sleep(1)

C:\Users\HP\AppData\Local\Temp\ipykernel_30060\3643148067.py:6: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  post_date = datetime.utcfromtimestamp(submission.created_utc)


Remaining requests: 999.0 | Resets in 37.98 seconds.
Remaining requests: 999.0 | Resets in 36.97 seconds.
Remaining requests: 999.0 | Resets in 35.97 seconds.
Remaining requests: 999.0 | Resets in 34.97 seconds.
Remaining requests: 999.0 | Resets in 33.97 seconds.
Remaining requests: 999.0 | Resets in 32.97 seconds.
Remaining requests: 999.0 | Resets in 31.97 seconds.
Remaining requests: 999.0 | Resets in 30.97 seconds.
Remaining requests: 999.0 | Resets in 29.97 seconds.
Remaining requests: 999.0 | Resets in 28.97 seconds.
Remaining requests: 999.0 | Resets in 27.97 seconds.
Remaining requests: 999.0 | Resets in 26.97 seconds.
Remaining requests: 999.0 | Resets in 25.97 seconds.
Remaining requests: 999.0 | Resets in 24.96 seconds.
Remaining requests: 999.0 | Resets in 23.96 seconds.
Remaining requests: 999.0 | Resets in 22.95 seconds.
Remaining requests: 999.0 | Resets in 21.94 seconds.
Remaining requests: 999.0 | Resets in 20.94 seconds.
Remaining requests: 999.0 | Resets in 19.94 se

In [33]:
post_df = pd.DataFrame(post_data)
post_df.to_csv('reddit_posts_PoliticalDiscusionsV2.csv', index=False)

In [35]:
post_df

,post_id,title,content,url,created_utc,subreddit,num_comments,upvotes,upvote_ratio,author
0,1gj0euk,What is the solution to the extreme polarizati...,It's apparent to everyone that political polar...,https://www.reddit.com/r/PoliticalDiscussion/c...,2024-11-03 23:25:51,PoliticalDiscussion,1,1,1.00,bambucks
1,1gixvv3,What do you think is going to be a big surpris...,"Like the title states, what do you think is go...",https://www.reddit.com/r/PoliticalDiscussion/c...,2024-11-03 21:33:46,PoliticalDiscussion,52,16,0.88,HERKFOOT21
2,1gio43r,The cost of the inability to successful comple...,The United States has a significant chance of ...,https://www.reddit.com/r/PoliticalDiscussion/c...,2024-11-03 14:30:25,PoliticalDiscussion,15,4,0.70,the_original_Retro
3,1gin9zc,What Will the Non MAGA GOP do if trump loses?,Will they have to form a new party since Lara ...,https://www.reddit.com/r/PoliticalDiscussion/c...,2024-11-03 13:50:36,PoliticalDiscussion,55,21,0.77,ExpensivePiece7560
4,1giooyn,What specific features of political systems yo...,While the issue of senate confirmation has got...,https://www.reddit.com/r/PoliticalDiscussion/c...,2024-11-03 14:57:15,PoliticalDiscussion,54,11,0.72,Awesomeuser90
...,...,...,...,...,...,...,...,...,...,...
861,1e4fpte,Which candidate REALLY has more troop support?,Both presidential candidates during the last p...,https://www.reddit.com/r/PoliticalDiscussion/c...,2024-07-16 04:29:27,PoliticalDiscussion,1,1,1.00,SeaworthinessNo6502
862,1e4dxbz,Is there a limit to charisma in politics?,"It seems charisma is one of, if not the most i...",https://www.reddit.com/r/PoliticalDiscussion/c...,2024-07-16 02:54:21,PoliticalDiscussion,41,0,0.50,hardfine
863,1e4br6p,Will Trump's plan of tariffs and tax cuts lowe...,With inflation being the #1 issue as stated by...,https://www.reddit.com/r/PoliticalDiscussion/c...,2024-07-16 01:07:32,PoliticalDiscussion,367,92,0.69,_awacz
864,1e48tjb,Does the US media have an accountability probl...,The right is critical of the left for propagan...,https://www.reddit.com/r/PoliticalDiscussion/c...,2024-07-15 22:53:18,PoliticalDiscussion,280,197,0.88,Tricky_Acanthaceae39
